# Generate dataset

this notebook use to generate the data for train model and test model

# Train dataset

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
FOLDER = '/Users/khunnoot/Desktop/year4/senior_project/solarmap_project/Data'
cloud_cm = 'ci_cloudmask.csv'
cloud_ov = 'ci_overview.csv'
DATASET_cm = 'DATASET_cloudmask.csv'
DATASET_ov = 'DATASET_overview.csv'

In [3]:
#load croped_img cloud mask dataset which contain 0-255 pixel with site and datetime
Image_CM = pd.read_csv(os.path.join(FOLDER,cloud_cm),parse_dates=True,index_col='Datetime')
Image_CM.index = Image_CM.index + pd.DateOffset(minutes=30) # + 30 min because of the lag time of Himawari images
Image_CM = Image_CM.reset_index()
Image_CM['Datetime']= Image_CM['Datetime'].astype(str)

#load croped_img RBG dataset which contain 0-255 pixel with site and datetime
Image_R = pd.read_csv(os.path.join(FOLDER,cloud_cm),parse_dates=True,index_col='Datetime')
Image_R.index = Image_R.index + pd.DateOffset(minutes=30)
Image_R = Image_R.reset_index()
Image_R['Datetime']= Image_R['Datetime'].astype(str)


#merge cloud mask layer and red channel layer in 'Image_CM_CHRED'
Image_CM_CHRED = pd.merge(Image_CM,Image_R,on=['Datetime','site_name'],how='right')

In [4]:
RED_ATT = pd.read_csv(os.path.join(FOLDER,DATASET_cm),parse_dates=True)
CM_ATT = pd.read_csv(os.path.join(FOLDER,DATASET_ov),parse_dates=True)

In [5]:
RED_ATT = RED_ATT[['Datetime','site_name','Iclr','I','k','HR','CI0','CI1']].copy()
RED_ATT.rename(columns={'CI0':'CI0_ov','CI1':'CI1_ov'},inplace=True)

CM_ATT = CM_ATT[['Datetime','site_name','Iclr','I','k','HR','CI0','CI1']].copy()
CM_ATT.rename(columns={'CI0':'CI0_cm','CI1':'CI1_cm'},inplace=True)

In [6]:
#merge attribute(ATT) from cloud mask channel and red channel 
ATT = pd.merge(CM_ATT[['Datetime','site_name','CI0_cm','CI1_cm']],RED_ATT,on=['Datetime','site_name'],how='inner')

#keep the intersec dataset from ATT and Images dataset
Image_2layers = pd.merge(ATT[['Datetime','site_name','I','Iclr','k']],Image_CM_CHRED,on=['Datetime','site_name'],how='inner')
Attribute_2layers = pd.merge(ATT,Image_CM_CHRED[['Datetime','site_name']],on=['Datetime','site_name'],how='inner')


In [13]:
Image_train = Image_2layers[((Image_2layers.site_name == 'ISL001'))|(Image_2layers.site_name == 'ISL033') |
              (Image_2layers.site_name == 'ISL036') | (Image_2layers.site_name == 'ISL042')]
Attribute_train = Attribute_2layers[((Attribute_2layers.site_name == 'ISL001'))|(Attribute_2layers.site_name == 'ISL033') |
              (Attribute_2layers.site_name == 'ISL036') | (Attribute_2layers.site_name == 'ISL042')]

In [18]:
Image_train.to_csv('Image_train.csv', index = False)
Attribute_train.to_csv('Attribute_train.csv', index = False)

# Test dataset

In [85]:
FOLDER_test = '/Users/khunnoot/Desktop/predict_irradiance/implementation/Data'
ci_cloudmask_test = 'ci_cloudmask_Synergy.csv'
ci_overview_test = 'ci_overview_Synergy.csv'

In [86]:
cm_test = pd.read_csv(os.path.join(FOLDER_test,ci_cloudmask_test), parse_dates = ['Datetime'], index_col = 'Datetime')
ov_test = pd.read_csv(os.path.join(FOLDER_test,ci_overview_test), parse_dates = ['Datetime'], index_col = 'Datetime')

In [87]:
# CI0 , CI1 are CI(t-1) , CI(t-2) respectively
cmlag = cm_test[['site_name','CI']].copy()
cmlag1 = cmlag.shift(freq='+30T') # for lag time merging
cmlag = cmlag.merge(cmlag1,on=['Datetime','site_name'],how='inner')
cmlag = cmlag.rename(columns={'CI_x':'CI0','CI_y':'CI1'})
cm_test = pd.merge(cm_test,cmlag, on = ['Datetime','site_name'], how = 'inner')

ovlag = ov_test[['site_name','CI']].copy()
ovlag1 = ovlag.shift(freq='+30T') # for lag time merging
ovlag = ovlag.merge(ovlag1,on=['Datetime','site_name'],how='inner')
ovlag = ovlag.rename(columns={'CI_x':'CI0','CI_y':'CI1'})
ov_test = pd.merge(ov_test,clag, on = ['Datetime','site_name'], how = 'inner')

In [88]:
cm_test = cm_test[['site_name','CI0','CI1']].copy()
cm_test.rename(columns={'CI0':'CI0_cm','CI1':'CI1_cm'},inplace=True)

ov_test = ov_test[['site_name','CI0','CI1']].copy()
ov_test.rename(columns={'CI0':'CI0_ov','CI1':'CI1_ov'},inplace=True)

In [91]:
Synergy_test = pd.merge(cm_test,ov_test, on = ['Datetime','site_name'], how = 'inner')

In [92]:
from SolarCalculator.clearskycal import calsolar

#coordinate of Synergy
Synergy_test['lat'] = 14.00294247371573
Synergy_test['lng'] = 100.6267303787081 
Synergy_test['alt'] = 5.0

In [93]:
#calculate zenith angle , Airmass, clear-sky irradiance and hour index
Synergy_test['zna'] = calsolar.cal_dzna(Synergy_test, Synergy_test.lat, Synergy_test.lng)
Synergy_test['AM'] = calsolar.cal_dairmass(Synergy_test, Synergy_test.lat, Synergy_test.lng)
Synergy_test['Iclr'] = calsolar.cal_dclearsky(Synergy_test, Synergy_test.lat, Synergy_test.lng,
                                             Synergy_test.alt, TL = 4.773765928887537)
Synergy_test['HR'] = Synergy_test.index.hour

In [94]:
Synergy_test = Synergy_test[['CI0_cm','CI1_cm','CI0_ov','CI1_ov','Iclr','HR']]

In [81]:
# Synergy_test.to_csv('Synergy_testset.csv')

In [98]:
Synergy_test

,Datetime,CI0_cm,CI1_cm,CI0_ov,CI1_ov,Iclr,HR
0,2023-03-08 08:20:00,0.023529,0.023529,0.341176,0.325490,327.238765,8
1,2023-06-24 09:10:00,0.360784,0.376471,0.560784,0.674510,636.343053,9
2,2023-05-24 14:40:00,0.035294,0.031373,0.278431,0.270588,771.130046,14
3,2023-04-21 18:40:00,0.000000,0.000000,0.031373,0.470588,0.000000,18
4,2023-03-30 16:40:00,0.066667,0.086275,0.431373,0.454902,334.519394,16
...,...,...,...,...,...,...,...
9296,2023-06-29 18:20:00,0.890196,0.921569,0.533333,0.776471,20.518187,18
9297,2023-04-30 07:40:00,0.337255,0.482353,0.000000,0.482353,301.773902,7
9298,2023-06-13 07:30:00,0.125490,0.113725,0.235294,0.274510,270.773465,7
9299,2023-02-08 09:10:00,0.007843,0.015686,0.325490,0.337255,443.313980,9
